<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#isnull" data-toc-modified-id="isnull-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>isnull</a></span><ul class="toc-item"><li><span><a href="#na" data-toc-modified-id="na-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>na</a></span></li><li><span><a href="#outer" data-toc-modified-id="outer-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>outer</a></span></li></ul></li></ul></div>

In [1]:
spark

25/09/16 15:27:25 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/09/16 15:27:25 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

In [3]:
meteoDataFrame  = spark.read.format('csv')\
    .option('sep',';')\
    .option('header','true')\
    .option('nullValue','mq')\
    .option('inferSchema', 'true')\
    .load('donnees/meteo/2024')\
    .cache()

schema = StructType([
        StructField('Id'           , StringType() , True),
        StructField('ville'        , StringType() , True),
        StructField('latitude'     , FloatType() , True),
        StructField('longitude'    , FloatType() , True),
        StructField('altitude'     , IntegerType() , True)])

villes  = spark.read.format('csv')   \
      .option('sep',';')                \
      .option('mergeSchema', 'true')    \
      .option('header','true')          \
      .schema(schema)                   \
      .load('donnees/postesSynop.csv')  \
      .cache()

@udf("string")
def formatVille(ville):
    if ville in ['CLERMONT-FD','MONT-DE-MARSAN',
                                   'ST-PIERRE','ST-BARTHELEMY METEO'] :
        return ville.title()
    else :
        if ville.find('-') != -1 :
            return ville[0:ville.find('-')].title()
        else:
            return ville.title()

villesT  = villes.select(
                col('Id').alias('id'),
                formatVille('ville').alias('ville'),
               'latitude',
               'longitude',
               'altitude')


meteo = meteoDataFrame.select(
                 col('numer_sta'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 coalesce( col('rr3'),
                           col('rr24')/8,
                           col('rr12')/4,
                           col('rr6')/2,
                           col('rr1')*3  ) )\
             .toDF('id','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression','precipitations')\
             .cache()

meteo.select('annee','mois','jour','temperature','humidite',
             'visibilite','pression').toPandas().head(3)

25/09/16 15:27:30 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/09/16 15:27:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/09/16 15:27:31 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/09/16 15:27:31 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
                                                                                

,annee,mois,jour,temperature,humidite,visibilite,pression
0,2024,5,1,13.1,0.94,14.20,100.13
1,2024,5,1,14.5,0.85,16.03,100.56
2,2024,5,1,8.8,0.88,NaN,100.74


In [4]:
villesT.toPandas().head(3)

,id,ville,latitude,longitude,altitude
0,07005,Abbeville,50.136002,1.834000,69
1,07015,Lille,50.570000,3.097500,47
2,07020,Pte De La Hague,49.725166,-1.939833,6


# isnull

In [5]:
meteo.where('id < 8000').count()
description = meteo.where('id < 8000')\
     .select('annee','temperature', 'humidite',
             'visibilite', 'pression','precipitations')\
     .describe()\
     .select([c if c == 'summary'
                else round(c,2).alias(c)
                for c in
                    ['summary','annee','temperature', 'humidite',
                   'visibilite', 'pression','precipitations']])\
     .toPandas()

In [6]:
meteo.where('id < 8000').where(meteo['temperature'].isNull()).count()

3042

In [7]:
meteo.where('id < 8000').where(meteo['humidite'].isNull()).count()

3346

In [8]:
meteo.where('id < 8000').where(meteo['visibilite'].isNull()).count()

10544

In [9]:
meteo.where('id < 8000').where(meteo['pression'].isNull()).count()

3432

In [10]:
meteo.where('id < 8000').where(meteo['precipitations'].isNull()).count()

3181

In [11]:
meteo.where('id < 8000').toPandas().isna().sum()

id                    0
annee                 0
mois                  0
jour                  0
mois_jour             0
temperature        3042
humidite           3346
visibilite        10544
pression           3432
precipitations     3181
dtype: int64

In [12]:
meteoDataFrame.toPandas().isna().sum()

numer_sta         0
date              0
pmer          10436
tend           3556
cod_tend       3551
dd              604
ff              604
t              5087
td             5789
u              4393
vv            45488
ww            48283
w1           157943
w2           166113
n            103947
nbas          60700
hbas          88204
cl           160254
cm           165059
ch           166898
pres           3466
niv_bar      170630
geop         170630
tend24         8783
tn12         144573
tn24         142652
tx12         145155
tx24         142449
tminsol       72479
sw           176468
tw           176468
raf10         28930
rafper         7078
per            6097
etat_sol     100346
ht_neige      77934
ssfrai       172045
perssfrai    172045
rr1            6731
rr3            8095
rr6           10091
rr12          11543
rr24          13603
phenspe1     176468
phenspe2     176468
phenspe3     176468
phenspe4     176468
nnuage1       86549
ctype1       159107
hnuage1       88945


In [1]:
import koalas as ks

ModuleNotFoundError: No module named 'koalas'

In [13]:

meteoDataFrame.to_koalas().isna().sum()

ModuleNotFoundError: No module named 'koalas'

In [ ]:
meteo.where('id < 8000')\
     .where(meteo['temperature'].isNotNull())\
     .where(meteo['humidite'].isNotNull() )\
     .where(meteo['visibilite'].isNotNull() )\
     .where(meteo['pression'].isNotNull() )\
     .count()

## na

In [ ]:
meteo.where('id < 8000')\
     .na.fill(0 ,["precipitations"])\
     .na.drop()\
     .count()

In [ ]:
meteo = meteoDataFrame.select(
                 col('numer_sta'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 col('rr1')*3,
                 col('rr3'),
                 col('rr6')/2,
                 col('rr12')/4,
                 col('rr24')/8)\
                   .toDF('id','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression',
                   'precipitations1','precipitations3','precipitations6',
                   'precipitations12','precipitations24')\
             .cache()

In [ ]:
meteo.where('id < 8000')\
       .select('precipitations1','precipitations3','precipitations6',
             'precipitations12','precipitations24','temperature')\
       .toDF('prec1','prec3','prec6','prec12','prec24','temp')\
       .describe()\
       .select([c if c == 'summary'
                else round(c,2).alias(c)
                for c in['prec1','prec3','prec6',
                          'prec12','prec24','temp']])\
      .toPandas()

In [ ]:
meteo.select( coalesce('precipitations3','precipitations24',
                       'precipitations12','precipitations6',
                       'precipitations1').alias('precipitations')).toPandas().head(20)

In [ ]:
meteo.where('id < 8000')\
     .select( coalesce('precipitations24','precipitations12',
                       'precipitations6','precipitations3',
                       'precipitations1').alias('precipitationsH')
            ).describe().toPandas()

In [ ]:
meteo.to_koalas().isna().sum()

In [ ]:
meteo.where('id < 8000').count()